Bank Marketing Campaign Data Analysis with DataFrame API


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
sc = SparkSession.builder.master('local').getOrCreate()

a) Load Bank Marketing Dataset and create DataFrame

In [4]:
df=sc.read.format('csv')\
    .option('header','True')\
    .option('delimiter',';')\
    .load('C:/Users/Miles/Documents/GitHub/futurense_hadoop-pyspark/labs/dataset/bankmarket/bankmarketdata.csv')
df.show(5)

+---+------------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age|         job|marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+------------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
| 58|  management|married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|
| 44|  technician| single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|
| 33|entrepreneur|married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|
| 47| blue-collar|married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92|       1|   -1|       0| unknown| no|
| 33|     unknown| single|  unknown|     no|      1|     no|  no|unknown|  5|  may|     19

b) Give marketing success rate. (No. of people subscribed / total no. of entries)

In [5]:
df.filter(df.y =='yes').count()/df.count()*100

11.698480458295547

c) Give marketing failure rate

In [6]:
df.filter(df.y =='no').count()/df.count()*100

88.30151954170445

d) Maximum, Mean, and Minimum age of the average targeted customer

In [7]:
df.agg({"age": "max"}).show()

+--------+
|max(age)|
+--------+
|      95|
+--------+



In [8]:
df.agg({"age": "min"}).show()

+--------+
|min(age)|
+--------+
|      18|
+--------+



e) Check the quality of customers by checking the average balance, median balance of customers

In [9]:
df.agg({'balance':'mean'}).show()

# df.groupBy("department").agg(func.max("age"), func.sum("expense"))

+------------------+
|      avg(balance)|
+------------------+
|1362.2720576850766|
+------------------+



f) Check if age matters in marketing subscription for deposit

In [14]:
print(df.filter(df.y == 'yes').groupBy('age').count().sort('count',ascending = False).show(10))

+---+-----+
|age|count|
+---+-----+
| 32|  221|
| 30|  217|
| 33|  210|
| 35|  209|
| 31|  206|
| 34|  198|
| 36|  195|
| 29|  171|
| 37|  170|
| 28|  162|
+---+-----+
only showing top 10 rows

None


g) Show AgeGroup [Teenagers, Youngsters, MiddleAgers, Seniors] wise Subscription Count.

In [19]:
age_cat_dict = {20:'Teen',40:'Young',60:'Middle',80:'Seniors',100:'Seniors'}
age_cat_udf = udf(lambda age: age_cat_dict[age],StringType())
df_age_cat = df.select('age','y').\
            filter(col('y') == 'yes').\
            withColumn('age_cat',age_cat_udf(ceil(df['age']/20)*20)).\
            groupBy('age_cat').count()
print(df_age_cat.show())



+-------+-----+
|age_cat|count|
+-------+-----+
|  Young| 2924|
+-------+-----+



h) Check if marital status mattered for subscription to deposit.

In [12]:
df.filter(col('y')=='yes').groupBy('marital').count().show()

+--------+-----+
| marital|count|
+--------+-----+
|divorced|  622|
| married| 2755|
|  single| 1912|
+--------+-----+



i) Check if age and marital status together mattered for subscription to deposit scheme

In [13]:
df.filter(col('y')=='yes').groupBy('age','marital').count().sort('count', ascending = False).show(15)

+---+-------+-----+
|age|marital|count|
+---+-------+-----+
| 30| single|  151|
| 28| single|  138|
| 29| single|  133|
| 32| single|  124|
| 26| single|  121|
| 34|married|  118|
| 31| single|  111|
| 27| single|  110|
| 35|married|  101|
| 36|married|  100|
| 25| single|   99|
| 37|married|   98|
| 33| single|   97|
| 33|married|   97|
| 32|married|   87|
+---+-------+-----+
only showing top 15 rows

